In [89]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [109]:
out='../output/'
inp='../input/monthly/'
pref='fpw'

In [110]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
744,(2021/05/31) 23-24,36.07,20
745,Min,32.58,0
746,Max,238.1,40
747,Average,76.052419354838709677419354839,12.936827956989247311827956989


In [111]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
740,(2021/05/31) 19-20,238.10,0.0
741,(2021/05/31) 20-21,86.00,0.0
742,(2021/05/31) 21-22,86.00,0.0
743,(2021/05/31) 22-23,36.07,20.0


In [112]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
740,(2021/05/31) 19-20,238.10,0.0,2021-05-31,19
741,(2021/05/31) 20-21,86.00,0.0,2021-05-31,20
742,(2021/05/31) 21-22,86.00,0.0,2021-05-31,21
743,(2021/05/31) 22-23,36.07,20.0,2021-05-31,22


In [113]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
740,238.10,0.0,2021-05-31,19
741,86.00,0.0,2021-05-31,20
742,86.00,0.0,2021-05-31,21
743,36.07,20.0,2021-05-31,22


In [114]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2021-05-31,238.10,0.0,19,2021,5,31,0
2021-05-31,86.00,0.0,20,2021,5,31,0
2021-05-31,86.00,0.0,21,2021,5,31,0


In [115]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()

daily

volume  day_of_week    value  price_weight_av  price_max  \
year month day                                                             
2017 3     6     200.0            0   5800.0            29.00      97.21   
           7     400.0           24  11600.0            29.00      97.21   
           8     400.0           48  11600.0            29.00      97.21   
           9     400.0           72  11600.0            29.00      97.21   
           10    200.0           80   5800.0            29.00      97.21   
...                ...          ...      ...              ...        ...   
2021 5     27     80.0           60   3037.6            37.97     180.00   
           28    160.0           96   6075.2            37.97     180.00   
           29    260.0          100   9872.2            37.97      85.00   
           30    400.0          120  15188.0            37.97      37.97   
           31     80.0            0   2885.6            36.07     238.10   

                price_min  price_mean  power_max  power_min  power_mean  \
year month day                                                            
2017 3     6        29.00   65.852500       50.0        0.0   10.000000   
           7        29.00   59.710417       50.0        0.0   16.666667   
           8        29.00   59.710417       50.0        0.0   16.666667   
           9        29.00   59.710417       50.0        0.0   16.666667   
           10       29.00   65.852500       50.0        0.0   10.000000   
...                   ...         ...        ...        ...         ...   
2021 5     27       37.97   99.344000       20.0        0.0    4.000000   
           28       37.97   89.115000       20.0        0.0    6.666667   
           29       37.97   54.430500       20.0        0.0   13.000000   
           30       37.97   37.970000       20.0       20.0   20.000000   
           31       36.07  114.039000       20.0        0.0    4.000000   

                price_morning_max  price_evening_max  price_standard  
year month day                                                        
2017 3     6                97.21              97.21           65.00  
           7                97.21              97.21           65.00  
           8                97.21              97.21           65.00  
           9                97.21              97.21           65.00  
           10               97.21              97.21           65.00  
...                           ...                ...             ...  
2021 5     27              180.00             180.00           85.00  
           28              180.00             180.00           85.00  
           29               85.00              85.00           37.97  
           30               37.97              37.97           37.97  
           31              238.10             238.10           86.00  

[1494 rows x 13 columns]

In [116]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='eve'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.0,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.0,off
2017-03-06,65.00,0.0,6,2017,3,6,0,0.0,off
2017-03-06,97.21,0.0,7,2017,3,6,0,0.0,off
2017-03-06,97.21,0.0,8,2017,3,6,0,0.0,off
...,...,...,...,...,...,...,...,...,...
2021-05-31,238.10,0.0,19,2021,5,31,0,0.0,off
2021-05-31,86.00,0.0,20,2021,5,31,0,0.0,off
2021-05-31,86.00,0.0,21,2021,5,31,0,0.0,off


In [117]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume  day_of_week    value  price_weight_av  price_max  \
year month day                                                             
2017 3     6     200.0            0   5800.0            29.00      97.21   
           7     400.0           24  11600.0            29.00      97.21   
           8     400.0           48  11600.0            29.00      97.21   
           9     400.0           72  11600.0            29.00      97.21   
           10    200.0           80   5800.0            29.00      97.21   
...                ...          ...      ...              ...        ...   
2021 5     27     80.0           60   3037.6            37.97     180.00   
           28    160.0           96   6075.2            37.97     180.00   
           29    260.0          100   9872.2            37.97      85.00   
           30    400.0          120  15188.0            37.97      37.97   
           31     80.0            0   2885.6            36.07     238.10   

                price_min  price_mean  power_max  power_min  power_mean  ...  \
year month day                                                           ...   
2017 3     6        29.00   65.852500       50.0        0.0   10.000000  ...   
           7        29.00   59.710417       50.0        0.0   16.666667  ...   
           8        29.00   59.710417       50.0        0.0   16.666667  ...   
           9        29.00   59.710417       50.0        0.0   16.666667  ...   
           10       29.00   65.852500       50.0        0.0   10.000000  ...   
...                   ...         ...        ...        ...         ...  ...   
2021 5     27       37.97   99.344000       20.0        0.0    4.000000  ...   
           28       37.97   89.115000       20.0        0.0    6.666667  ...   
           29       37.97   54.430500       20.0        0.0   13.000000  ...   
           30       37.97   37.970000       20.0       20.0   20.000000  ...   
           31       36.07  114.039000       20.0        0.0    4.000000  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2017 3     6                   NaN                 NaN       200.0   
           7                   NaN                 NaN       400.0   
           8                   NaN                 NaN       400.0   
           9                   NaN                 NaN       400.0   
           10                  NaN                 NaN       200.0   
...                            ...                 ...         ...   
2021 5     27                  NaN                 NaN        80.0   
           28                  NaN                 NaN       160.0   
           29                  NaN                 NaN       260.0   
           30                  NaN                 NaN       400.0   
           31                  NaN                 NaN        80.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2017 3     6                NaN             NaN             NaN     5800.0   
           7                NaN             NaN             NaN    11600.0   
           8                NaN             NaN             NaN    11600.0   
           9                NaN             NaN             NaN    11600.0   
           10               NaN             NaN             NaN     5800.0   
...                         ...             ...             ...        ...   
2021 5     27               NaN             NaN             NaN     3037.6   
           28               NaN             NaN             NaN     6075.2   
           29               NaN             NaN             NaN     9872.2   
           30               NaN             NaN             NaN    15188.0   
           31               NaN             NaN             NaN     2885.6   

                standard_value  morning_value  evening_value

In [118]:
daily.to_csv(out + pref + '_daily.csv')

In [119]:
hourly.to_csv(out + pref + '_hourly.csv')

In [120]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,64.842105,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,...,0.0,30.905773,30.905773,NaN,NaN,NaN,25.345982,NaN,NaN,NaN
1,2017,4,1148.500000,67.000000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,...,0.0,33.403017,33.403017,NaN,NaN,NaN,49.934783,NaN,NaN,NaN
2,2017,5,959.395238,70.285714,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,...,0.0,44.272995,44.272995,NaN,NaN,NaN,40.949797,NaN,NaN,NaN
3,2017,6,480.730769,66.000000,23358.136923,71.232500,29.95,58.039212,28.461538,10.423077,...,0.0,48.588812,48.588812,NaN,NaN,NaN,20.557566,NaN,NaN,NaN
4,2017,7,462.677419,69.806452,20526.554677,70.355000,25.05,52.592774,22.838710,11.580645,...,0.0,44.364721,44.364721,NaN,NaN,NaN,20.144663,NaN,NaN,NaN
5,2017,8,457.580645,66.451613,22350.982258,64.280500,28.61,51.959191,23.419355,14.354839,...,0.0,48.845996,48.845996,NaN,NaN,NaN,19.922753,NaN,NaN,NaN
6,2017,9,437.583333,71.466667,19796.830000,63.146000,26.93,50.736536,21.000000,12.583333,...,0.0,45.241280,45.241280,NaN,NaN,NaN,18.970376,NaN,NaN,NaN
7,2017,10,637.109677,66.580645,29922.428645,60.922500,26.92,48.886422,29.032258,22.554839,...,0.0,46.965899,46.965899,NaN,NaN,NaN,27.739326,NaN,NaN,NaN
8,2017,11,660.423333,69.200000,28101.312300,59.438500,26.14,48.175603,38.596667,22.306667,...,0.0,42.550453,42.550453,NaN,NaN,NaN,28.631069,NaN,NaN,NaN
9,2017,12,624.487097,73.032258,28725.856194,64.220000,26.61,49.478970,32.809677,20.435484,...,0.0,45.999119,45.999119,NaN,NaN,NaN,27.189747,NaN,NaN,NaN


In [121]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,day_of_week,value_mean,price_max,price_min,price_mean,power_max,power_min,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,64.842105,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,...,0.0,30.905773,30.905773,NaN,NaN,NaN,25.345982,NaN,NaN,NaN
1,2017,4,1148.500000,67.000000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,...,0.0,33.403017,33.403017,NaN,NaN,NaN,49.934783,NaN,NaN,NaN
2,2017,5,959.395238,70.285714,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,...,0.0,44.272995,44.272995,NaN,NaN,NaN,40.949797,NaN,NaN,NaN
3,2017,6,480.730769,66.000000,23358.136923,71.232500,29.95,58.039212,28.461538,10.423077,...,0.0,48.588812,48.588812,NaN,NaN,NaN,20.557566,NaN,NaN,NaN
4,2017,7,462.677419,69.806452,20526.554677,70.355000,25.05,52.592774,22.838710,11.580645,...,0.0,44.364721,44.364721,NaN,NaN,NaN,20.144663,NaN,NaN,NaN
5,2017,8,457.580645,66.451613,22350.982258,64.280500,28.61,51.959191,23.419355,14.354839,...,0.0,48.845996,48.845996,NaN,NaN,NaN,19.922753,NaN,NaN,NaN
6,2017,9,437.583333,71.466667,19796.830000,63.146000,26.93,50.736536,21.000000,12.583333,...,0.0,45.241280,45.241280,NaN,NaN,NaN,18.970376,NaN,NaN,NaN
7,2017,10,637.109677,66.580645,29922.428645,60.922500,26.92,48.886422,29.032258,22.554839,...,0.0,46.965899,46.965899,NaN,NaN,NaN,27.739326,NaN,NaN,NaN
8,2017,11,660.423333,69.200000,28101.312300,59.438500,26.14,48.175603,38.596667,22.306667,...,0.0,42.550453,42.550453,NaN,NaN,NaN,28.631069,NaN,NaN,NaN
9,2017,12,624.487097,73.032258,28725.856194,64.220000,26.61,49.478970,32.809677,20.435484,...,0.0,45.999119,45.999119,NaN,NaN,NaN,27.189747,NaN,NaN,NaN


In [122]:
monthly.to_csv(out + pref + '_monthly.csv')

In [123]:
monthly_filt=monthly.loc[monthly['year']<2020]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']<2020]

In [124]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]

calmonthly

,month,day_of_week,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,off_value,standard_value,morning_value,evening_value,volume,off_volume,standard_volume,morning_volume,evening_volume,value
0,1,65.806452,3773.648435,40.405970,95.004839,27.899355,53.172812,5.358065,2.546774,3.714879,...,1.169831e+05,0.0,0.0,0.0,2656.750000,2656.750000,0.0,0.0,0.0,1.169831e+05
1,2,70.612245,2417.153388,56.560000,102.117347,28.066122,66.069820,9.685714,0.000000,2.876888,...,5.922026e+04,0.0,0.0,0.0,1627.100000,1627.100000,0.0,0.0,0.0,5.922026e+04
2,3,70.617284,8292.535185,27.453025,76.193457,26.825802,49.678801,21.814815,4.456790,11.275720,...,2.238985e+05,0.0,0.0,0.0,7232.000000,7232.000000,0.0,0.0,0.0,2.238985e+05
3,4,67.062500,10068.978281,30.693082,68.357812,27.090313,46.758288,20.775000,4.718750,11.604297,...,2.148049e+05,0.0,0.0,0.0,5590.000000,5590.000000,0.0,0.0,0.0,2.148049e+05
4,5,68.578313,15639.570108,38.174440,74.521807,29.157349,49.104466,25.832530,8.522892,14.574056,...,4.326948e+05,0.0,0.0,0.0,9485.100000,9485.100000,0.0,0.0,0.0,4.326948e+05
5,6,70.046512,19799.947163,42.372336,113.796512,32.144884,64.339472,37.974419,7.417442,19.784041,...,5.675985e+05,0.0,0.0,0.0,12912.200000,12912.200000,0.0,0.0,0.0,5.675985e+05
6,7,67.182796,21391.251355,45.643103,125.730538,35.008387,71.012225,36.073118,7.515054,19.550448,...,6.631288e+05,0.0,0.0,0.0,14032.466667,14032.466667,0.0,0.0,0.0,6.631288e+05
7,8,69.247312,21718.561763,55.064815,131.696882,35.502688,71.143965,27.655914,7.647312,17.298790,...,6.732754e+05,0.0,0.0,0.0,12375.466667,12375.466667,0.0,0.0,0.0,6.732754e+05
8,9,70.577778,33599.931111,51.947519,92.956000,30.823556,54.781711,54.683333,12.066667,30.563565,...,1.007998e+06,0.0,0.0,0.0,21205.833333,21205.833333,0.0,0.0,0.0,1.007998e+06
9,10,66.064516,19082.346978,48.126553,83.135914,29.961613,53.965246,19.497849,10.026882,17.083799,...,5.915528e+05,0.0,0.0,0.0,12249.833333,12249.833333,0.0,0.0,0.0,5.915528e+05


In [125]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [126]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      38.381087  11.568478   509.468380
      1      38.381087  11.568478   509.468380
      2      38.381087  11.568478   509.468380
      3      38.381087  11.568478   509.468380
      4      38.316935  11.626613   513.657411
...                ...        ...          ...
12    19    121.692500  14.888710   853.341532
      20     65.951855  25.118548  1396.648806
      21     65.951855  25.118548  1396.648806
      22     35.736452  46.086290  1868.078177
      23     35.736452  46.086290  1868.078177

[288 rows x 3 columns]

In [127]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')